In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MO_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2660988802,MO,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,3162008008,MO,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,4904497002,MO,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,5467867110,MO,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14
4,5529817004,MO,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2660988802,MO,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,3162008008,MO,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,4904497002,MO,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,5467867110,MO,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14
4,5529817004,MO,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
98,5313117007,MO,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,NaN
172,5855317002,MO,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,NaN
363,6800117009,MO,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,NaN
865,1232848500,MO,"391 LANDON RD, BOURBON, MO",38.182280,-91.208434,NaN
1696,3975347104,MO,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,NaN
1837,1408137105,MO,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,NaN
2668,3707268300,MO,"28406.00, Moberly, MO",39.418369,-92.438237,NaN
3118,2598218607,MO,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,NaN
3192,6533167007,MO,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,NaN
3485,7904067104,MO,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('29')] #MO
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/42 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

42


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10329/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
98,5313117007,MO,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,[]
172,5855317002,MO,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,[]
363,6800117009,MO,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,[]
865,1232848500,MO,"391 LANDON RD, BOURBON, MO",38.182280,-91.208434,[290554501041064]
1696,3975347104,MO,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,[]
1837,1408137105,MO,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,[]
2668,3707268300,MO,"28406.00, Moberly, MO",39.418369,-92.438237,[291754905002015]
3118,2598218607,MO,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,[]
3192,6533167007,MO,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,[]
3485,7904067104,MO,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10329/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
98,5313117007,MO,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,NaN
172,5855317002,MO,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,NaN
363,6800117009,MO,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,NaN
865,1232848500,MO,"391 LANDON RD, BOURBON, MO",38.182280,-91.208434,290554501041064
1696,3975347104,MO,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,NaN
1837,1408137105,MO,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,NaN
2668,3707268300,MO,"28406.00, Moberly, MO",39.418369,-92.438237,291754905002015
3118,2598218607,MO,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,NaN
3192,6533167007,MO,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,NaN
3485,7904067104,MO,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
98,5313117007,MO,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,NaN
172,5855317002,MO,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,NaN
363,6800117009,MO,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,NaN
1696,3975347104,MO,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,NaN
1837,1408137105,MO,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,NaN
3118,2598218607,MO,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,NaN
3192,6533167007,MO,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,NaN
3485,7904067104,MO,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,NaN
4328,7509317009,MO,"115 S 5th St, COLUMBIA, MO",40.895588,-80.693220,NaN
4908,9542958304,MO,"1601 E Highway 84, Hayti, MO",31.522888,-90.580611,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
98,5313117007,NaN
172,5855317002,NaN
363,6800117009,NaN
865,1232848500,290554501041064
1696,3975347104,NaN
1837,1408137105,NaN
2668,3707268300,291754905002015
3118,2598218607,NaN
3192,6533167007,NaN
3485,7904067104,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MO_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,496374,2660988802,MO,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,496375,3162008008,MO,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,496376,4904497002,MO,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,496377,5467867110,MO,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14
4,496378,5529817004,MO,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,496374,2660988802,MO,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14,NaN
1,496375,3162008008,MO,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14,NaN
2,496376,4904497002,MO,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14,NaN
3,496377,5467867110,MO,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14,NaN
4,496378,5529817004,MO,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14,NaN
...,...,...,...,...,...,...,...,...
16334,512708,9713238502,MO,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,2.916503e+14,NaN
16335,512709,9761878500,MO,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,2.916503e+14,NaN
16336,512710,9774858406,MO,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,2.904702e+14,NaN
16337,512711,9851497109,MO,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,2.909501e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,496374,2660988802,MO,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14,NaN,291892150011052.0
1,496375,3162008008,MO,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14,NaN,295101278002000.0
2,496376,4904497002,MO,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14,NaN,290470221006106.0
3,496377,5467867110,MO,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14,NaN,290770042021046.0
4,496378,5529817004,MO,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14,NaN,290950153003025.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2660988802,MO,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,291892150011052.0
1,3162008008,MO,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,295101278002000.0
2,4904497002,MO,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,290470221006106.0
3,5467867110,MO,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,290770042021046.0
4,5529817004,MO,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,290950153003025.0
...,...,...,...,...,...,...
16334,9713238502,MO,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,291650306011020.0
16335,9761878500,MO,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,291650303091000.0
16336,9774858406,MO,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,290470202022009.0
16337,9851497109,MO,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,290950081002015.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
98,5313117007,MO,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,NaN
172,5855317002,MO,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,NaN
363,6800117009,MO,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,NaN
1696,3975347104,MO,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,NaN
1837,1408137105,MO,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,NaN
3118,2598218607,MO,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,NaN
3192,6533167007,MO,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,NaN
3485,7904067104,MO,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,NaN
4328,7509317009,MO,"115 S 5th St, COLUMBIA, MO",40.895588,-80.693220,NaN
4908,9542958304,MO,"1601 E Highway 84, Hayti, MO",31.522888,-90.580611,NaN


Convert to File

In [19]:
fips_MO = fips_merge_drop

In [20]:
fips_MO.to_csv("../../../data/state_data/geo/geo_fips/23/MO_fips_scraped.csv") 